In [10]:
import os
import pandas as pd

In [11]:
path = os.path.expanduser("~/Documents/Work_CBMSO/Testing_Leishmania_project/11.True_Positive_coorectCoorCSV")
os.chdir(path)
os.getcwd()

'/home/rfpacheco/Documents/Work_CBMSO/Testing_Leishmania_project/11.True_Positive_coorectCoorCSV'

Prepare SIDERs data for each cromosome:

In [12]:
import glob
import re

In [13]:
# Path to BLAST results
results_files = glob.glob("./data/SIDERs/*.fasta")
results_files

# Let's order them by the cromosome number
pattern = re.compile(r"Chr(\d+)")
results_files = sorted(results_files, key=lambda x: int(pattern.search(x).group(1)))

# Let's create a dictionary with the chromosome number as key and the file path as value
files_dict = {}
for i in range(len(results_files)):
    files_dict[f"LinJ.{str(i+1).zfill(2)}"] = results_files[i]

Prepare the genome for each cromosome:

In [14]:
from Bio import SeqIO

In [15]:
genome_path = "./data/dict/Whole_genome/TriTrypDB-67_LinfantumJPCM5_Genome.fasta"
folder_path = "./data/dict"

chromosomes_dict = {}
for record in SeqIO.parse(genome_path, "fasta"):
    chr_folder_path = os.path.join(folder_path, record.id)
    os.makedirs(chr_folder_path, exist_ok=True)
    output_file = f"{record.id}.fasta"
    output_path = os.path.join(chr_folder_path, output_file)
    chromosomes_dict[record.id] = output_path
    with open(output_path, "w") as f:
        f.write(f">{record.id}\n{record.seq}\n")

Create a BLASTn dict in each cromosome file

In [16]:
import subprocess

In [17]:
for key, value in chromosomes_dict.items():
    cmd = f"makeblastdb -in {value} -dbtype nucl -parse_seqids -out {value}"
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

Now let's start making the BLASTn to find the coordinates

In [18]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def get_fasta_format(sequence):
    seq = Seq(sequence)
    seq_record = SeqRecord(seq)
    fasta_format = seq_record.format("fasta")
    print(fasta_format)

In [19]:
def columns_to_numeric(data_input, columns_to_convert = ["pident", "length", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qlen", "slen"]):
    for column in columns_to_convert:
        data_input[column] = pd.to_numeric(data_input[column], errors='coerce')
    return data_input

In [20]:
def blastn_blaster(query_path, dict_path, perc_identity):
    cmd = "blastn -word_size 15 -query " \
        + query_path + " -db " \
        + dict_path \
        + " -perc_identity " + str(perc_identity) \
        + " -outfmt '10 qseqid sseqid pident length qstart qend sstart send evalue bitscore qlen slen sstrand sseq'"
    data = subprocess.check_output(cmd, shell=True, universal_newlines=True)  # Important the E value
    data = pd.DataFrame([x.split(",") for x in data.split("\n") if x])
    data.columns = ["qseqid", "sseqid", "pident", "length", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qlen", "slen", "sstrand", "sseq"]
    return data

In [21]:
whole_SIDERs = pd.DataFrame()
for key, value in files_dict.items():
    print("")
    print(f"{key}:")
    cmd = f"cat {value} | grep '^>' | wc -l"
    n = subprocess.check_output(cmd, shell=True, universal_newlines=True)
    print(f"\t- Number of SIDERs: {n}")

    data = blastn_blaster(value, chromosomes_dict[key], 0)
    data = columns_to_numeric(data)
    print(f"\t- Number of hits: {data.shape[0]}")
    print(f"\t- Number of unique hits: {data['qseqid'].nunique()}")

    data_filtered = data[data["length"] == data["qlen"]]
    print(f"\t- Number of hits where length == qlen: {data_filtered.shape[0]}")
    print(f"\t\t- Unique hits {data_filtered['qseqid'].nunique()}")

    data_filtered2 = data_filtered[data_filtered["pident"] >= 99.00]
    print(f"\t- Number of hits where pident == 100.00: {data_filtered2.shape[0]}")
    print(f"\t\t- Unique hits {data_filtered2['qseqid'].nunique()}")

    if data_filtered2['qseqid'].nunique() == int(n) and data_filtered2.shape[0] == int(n):
        print(f"\t- Original SIDERs == Hits: TRUE")
    elif data_filtered2['qseqid'].nunique() == int(n) and data_filtered2.shape[0] > int(n):
        print(f"\t- Original SIDERs == Hits: TRUE +")
    else:
        print(f"\t- Original SIDERs == Hits: FALSE")
    
    whole_SIDERs = pd.concat([whole_SIDERs, data_filtered], ignore_index=True)


LinJ.01:
	- Number of SIDERs: 7

	- Number of hits: 86
	- Number of unique hits: 7
	- Number of hits where length == qlen: 7
		- Unique hits 7
	- Number of hits where pident == 100.00: 7
		- Unique hits 7
	- Original SIDERs == Hits: TRUE

LinJ.02:
	- Number of SIDERs: 9

	- Number of hits: 60
	- Number of unique hits: 9
	- Number of hits where length == qlen: 18
		- Unique hits 9
	- Number of hits where pident == 100.00: 14
		- Unique hits 9
	- Original SIDERs == Hits: TRUE +

LinJ.03:
	- Number of SIDERs: 8

	- Number of hits: 24
	- Number of unique hits: 8
	- Number of hits where length == qlen: 8
		- Unique hits 8
	- Number of hits where pident == 100.00: 8
		- Unique hits 8
	- Original SIDERs == Hits: TRUE

LinJ.04:
	- Number of SIDERs: 15

	- Number of hits: 93
	- Number of unique hits: 15
	- Number of hits where length == qlen: 18
		- Unique hits 15
	- Number of hits where pident == 100.00: 15
		- Unique hits 15
	- Original SIDERs == Hits: TRUE

LinJ.05:
	- Number of SIDERs: 15


In [22]:
whole_SIDERs.shape

(1606, 14)

In [23]:
whole_SIDERs.head()

,qseqid,sseqid,pident,length,qstart,qend,sstart,send,evalue,bitscore,qlen,slen,sstrand,sseq
0,1-24471d,LinJ.01,100.000,660,1,660,24099,24758,0.000000e+00,1219.0,660,278267,plus,GGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCATCCCAGG...
1,1-36305r,LinJ.01,99.517,621,1,621,35933,35313,0.000000e+00,1131.0,621,278267,minus,GGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCATCCCAGG...
2,1-55910r,LinJ.01,99.846,650,1,650,55533,54885,0.000000e+00,1194.0,650,278267,minus,CGGAAGGCACGCACCTCCATGCGTGGCATCCCAGGGTCCAGCGCCC...
3,1-112979r,LinJ.01,99.614,259,1,259,114017,113760,5.440000e-135,472.0,259,278267,minus,GGGGTGAGGCGGCGGCGCACAGACACACACACACACACACACACGC...
4,1-129494r,LinJ.01,100.000,447,1,447,130577,130131,0.000000e+00,826.0,447,278267,minus,TGATGACGGGGCGGCGGCGCACAGACACACACACACACACACACAC...


In [24]:
whole_SIDERs.sort_values(by=["sseqid", "qseqid"], inplace=True)

In [26]:
whole_SIDERs.to_csv("TP_SIDERs_coor.csv", sep=",", header=True, index=False)

In [468]:
data = blastn_blaster(files_dict["LinJ.35"], chromosomes_dict["LinJ.35"], 0)
data = columns_to_numeric(data)
print(data.shape)

(2780, 14)


In [450]:
data["qseqid"].nunique()

71

In [451]:
data["qseqid"].value_counts()

qseqid
35M-1676847d      67
35M-1693985r      67
35N-1818131d      64
35B-385760d       64
35N-1830577d      64
                  ..
35-260414d        15
35K-1390003r      14
35B-364332d       14
19D-35-858746d    14
35A-2072704r      14
Name: count, Length: 71, dtype: int64

In [452]:
data_2 = data[data["length"] == data["qlen"]]
print(data_2.shape)
data_2.head()

(104, 14)


,qseqid,sseqid,pident,length,qstart,qend,sstart,send,evalue,bitscore,qlen,slen,sstrand,sseq
0,35A-41620r,LinJ.35,100.0,487,1,487,47291,46805,0.0,900.0,487,2019672,minus,CCCACTCTGTATGGGAAAGCCATACAGCCCCCTACTTGCTGACAAA...
50,35A-58693d,LinJ.35,100.0,502,1,502,64364,64865,0.0,928.0,502,2019672,plus,CCCGCTCTGTGTGGGAAAGCCATACAGCCCCCTACTTGCTGACAAA...
100,35-103785r,LinJ.35,100.0,588,1,588,109732,109145,0.0,1086.0,588,2019672,minus,TGATGGCGGGAGACACATACGTGCGTGATATCTCTCAGCGTCCAGT...
119,35-260414d,LinJ.35,100.0,559,1,559,193800,194358,0.0,1033.0,559,2019672,plus,CTCAGTGCGTGATATCTCAGCGTCCAGTGCACCCCCACACACACAC...
134,35-315582d,LinJ.35,100.0,578,1,578,248724,249301,0.0,1068.0,578,2019672,plus,GGAGGTGACACACCTCAGCGCATGGTATCCAGGCTCCAATGCCCCC...


In [453]:
data_2["qseqid"].nunique()

69

In [454]:
data_2["qseqid"].value_counts()

qseqid
35F-2074715r    6
35O-1891302d    5
35A-2072704r    3
35A-2069688r    3
35C-416008d     3
               ..
35-1185665r     1
35J-1320100r    1
35J-1355845r    1
35K-1390003r    1
35F-724657d     1
Name: count, Length: 69, dtype: int64

In [455]:
data["qseqid"].value_counts().index.difference(data_2["qseqid"].value_counts().index)


Index(['19D-35-858746d', '35F-659109r'], dtype='object', name='qseqid')

In [465]:
get_fasta_format(data.iloc[787]["sseq"].replace("-", ""))

><unknown id> <unknown description>
TGTGTGTGTGTGTGTGGAGGCCCAGCAGCACCCCCTATCCCTGTCAATGCCGAGCCACTT
GCGGTGGTGACGCGGGGCCAGGCACCCGCGACGTGGTGGTGCCAGAGCGCTTCATCGCTG
CTCATGCCGGCAGCCATGTCGTGGACGGCGTTGCGTCGGAGCGGCCTGCGACAGCGAGCA
CGCG



In [469]:
data[data["qseqid"] == "35F-659109r"]

,qseqid,sseqid,pident,length,qstart,qend,sstart,send,evalue,bitscore,qlen,slen,sstrand,sseq
787,35F-659109r,LinJ.35,100.000,184,1,184,602908,602725,7.900000e-95,340.0,184,2019672,minus,TGTGTGTGTGTGTGTGGAGGCCCAGCAGCACCCCCTATCCCTGTCA...
788,35F-659109r,LinJ.35,99.457,184,1,184,648420,648237,3.680000e-93,335.0,184,2019672,minus,TGTGTGTGTGTGTGTGGAGGCCCAGCAGCACCCCCTATCCCTGCCA...
789,35F-659109r,LinJ.35,98.913,184,1,184,654654,654471,1.710000e-91,329.0,184,2019672,minus,TGTGTGTGTGTGTGTGGAGGCCAAGCAGCACCCCCTATCCCTGCCA...
790,35F-659109r,LinJ.35,98.913,184,1,184,679825,680008,1.710000e-91,329.0,184,2019672,plus,TGTGTGTGTGTGTGTGGAGGCCAAGCAGCACCCCCTATCCCTGCCA...
791,35F-659109r,LinJ.35,98.370,184,1,184,633286,633103,7.960000e-90,324.0,184,2019672,minus,TGTGTGTGTGTGTGTGGAGGCCAAGCAGCACCCCCTATCCCTGCCA...
792,35F-659109r,LinJ.35,98.370,184,1,184,668514,668697,7.960000e-90,324.0,184,2019672,plus,TGTGTGTGTGTGTGTGGAGGCCAAGCAGCACCCCCTATCCCTGCCA...
793,35F-659109r,LinJ.35,74.118,170,17,174,1721627,1721795,8.910000e-10,58.4,184,2019672,plus,GAGGCCAAGCAGCCCCTCCTCCATCCCCTGCCAATGCAGCGCTGCA...
794,35F-659109r,LinJ.35,73.964,169,17,174,1744563,1744731,8.910000e-10,58.4,184,2019672,plus,GAGGCCAAGCAGCTCCTCCTCCATCCCCTGCCAATGCAGCGCTGCA...
795,35F-659109r,LinJ.35,73.373,169,17,174,1771644,1771812,4.150000e-08,52.8,184,2019672,plus,GAAGCCAAGCAGCCCCTCCTCCATCCCCTGCCAATGCAGCGCTGCA...
796,35F-659109r,LinJ.35,73.373,169,17,174,1784090,1784258,4.150000e-08,52.8,184,2019672,plus,GAAGCCAAGCAGCCCCTCCTCCATCCCCTGCCAATGCAGCGCTGCA...


In [458]:
duplicate_qseqids = data_2["qseqid"].value_counts()[lambda x: x > 1].index
filtered_data_2 = data_2[data_2["qseqid"].isin(duplicate_qseqids)]
filtered_data_2

,qseqid,sseqid,pident,length,qstart,qend,sstart,send,evalue,bitscore,qlen,slen,sstrand,sseq
298,35C-416008d,LinJ.35,100.000,421,1,421,349148,349568,0.000000e+00,778.0,421,2019672,plus,GGAAGCCATGCGGCCCCTCCCCCTTATCCCTGCCAATGCCGAAGCA...
299,35C-416008d,LinJ.35,99.287,421,1,421,360979,360559,0.000000e+00,761.0,421,2019672,minus,GGACGCCAAGCGGCCCCTCCCCCTTATCCCTGCCAATGCCGAAGCA...
300,35C-416008d,LinJ.35,99.050,421,1,421,409597,409177,0.000000e+00,756.0,421,2019672,minus,GGACGCCAAGCGGCCCCTCCCCCTTATCCCTGCCAATGCCGAAGCA...
343,35C-427900r,LinJ.35,100.000,494,1,494,361039,360546,0.000000e+00,913.0,494,2019672,minus,GACGACGAGGACTACCCCAGCGTGACATCACAGGGTCCAGCGCCCT...
344,35C-427900r,LinJ.35,99.190,494,1,494,409657,409164,0.000000e+00,891.0,494,2019672,minus,GACGACGAGGACTACCCCAGCGTGACATCACAGGGTCCAGCGCCCT...
536,35-498559r,LinJ.35,100.000,567,1,567,431694,431128,0.000000e+00,1048.0,567,2019672,minus,ATGACGGGGGATACTTCATTACGTGGCATCACAGGGTCCAGCGCCC...
537,35-498559r,LinJ.35,99.824,567,1,567,436889,436323,0.000000e+00,1042.0,567,2019672,minus,ATGACGGGGGATACTTCATTACGTGGCATCACAGGGTCCAGCGCCC...
585,35D-544895d,LinJ.35,100.000,518,1,518,483224,483741,0.000000e+00,957.0,518,2019672,plus,AGGCTCCAGTACACCCGCTCTGTGTGGAGAGGTCAAGCAGCCACTC...
586,35D-544895d,LinJ.35,99.807,518,1,518,492087,491570,0.000000e+00,952.0,518,2019672,minus,AGGGTCCAGTACACCCGCTCTGTGTGGAGAGGTCAAGCAGCCACTC...
629,35D-553758r,LinJ.35,100.000,518,1,518,492087,491570,0.000000e+00,957.0,518,2019672,minus,AGGGTCCAGTACACCCGCTCTGTGTGGAGAGGTCAAGCAGCCACTC...


In [459]:
unique_qseqids = data_2["qseqid"].value_counts()[lambda x: x == 1].index
filtered_unique_data_2 = data_2[data_2["qseqid"].isin(unique_qseqids)]
filtered_unique_data_2

,qseqid,sseqid,pident,length,qstart,qend,sstart,send,evalue,bitscore,qlen,slen,sstrand,sseq
0,35A-41620r,LinJ.35,100.0,487,1,487,47291,46805,0.000000e+00,900.0,487,2019672,minus,CCCACTCTGTATGGGAAAGCCATACAGCCCCCTACTTGCTGACAAA...
50,35A-58693d,LinJ.35,100.0,502,1,502,64364,64865,0.000000e+00,928.0,502,2019672,plus,CCCGCTCTGTGTGGGAAAGCCATACAGCCCCCTACTTGCTGACAAA...
100,35-103785r,LinJ.35,100.0,588,1,588,109732,109145,0.000000e+00,1086.0,588,2019672,minus,TGATGGCGGGAGACACATACGTGCGTGATATCTCTCAGCGTCCAGT...
119,35-260414d,LinJ.35,100.0,559,1,559,193800,194358,0.000000e+00,1033.0,559,2019672,plus,CTCAGTGCGTGATATCTCAGCGTCCAGTGCACCCCCACACACACAC...
134,35-315582d,LinJ.35,100.0,578,1,578,248724,249301,0.000000e+00,1068.0,578,2019672,plus,GGAGGTGACACACCTCAGCGCATGGTATCCAGGCTCCAATGCCCCC...
175,35-336806d,LinJ.35,100.0,587,1,587,269955,270541,0.000000e+00,1085.0,587,2019672,plus,CCCATTGATGACGGGGAACACCTCACTGTTGCACCTCAGGGTTCAT...
220,35B-364332d,LinJ.35,100.0,453,1,453,297479,297931,0.000000e+00,837.0,453,2019672,plus,CCCACTCTCAGTGTGGAGGGAAAGCCACGCAGCCCCCTACCCCCTA...
234,35B-385760d,LinJ.35,100.0,522,1,522,318904,319425,0.000000e+00,965.0,522,2019672,plus,CCCGCTCTCTGTGAGGGGGGAAGTCGAGCAGCCCCCTACCCCCTAT...
395,35C-452591d,LinJ.35,100.0,527,1,527,385729,386255,0.000000e+00,974.0,527,2019672,plus,GATGACGAGGACTACCCCAGCGTGACATCACAGGGTCCAGCGCCCT...
451,35-473451r,LinJ.35,100.0,351,1,351,406586,406236,0.000000e+00,649.0,351,2019672,minus,GCCCCTTCTTTTCCCCGGCATTTGTTGAGCTCTGATGACGAGGAAC...


In [460]:
data_3 = data_2[data_2["pident"] == 100.00]
print(data_3.shape)
data_3.head()

(72, 14)


,qseqid,sseqid,pident,length,qstart,qend,sstart,send,evalue,bitscore,qlen,slen,sstrand,sseq
0,35A-41620r,LinJ.35,100.0,487,1,487,47291,46805,0.0,900.0,487,2019672,minus,CCCACTCTGTATGGGAAAGCCATACAGCCCCCTACTTGCTGACAAA...
50,35A-58693d,LinJ.35,100.0,502,1,502,64364,64865,0.0,928.0,502,2019672,plus,CCCGCTCTGTGTGGGAAAGCCATACAGCCCCCTACTTGCTGACAAA...
100,35-103785r,LinJ.35,100.0,588,1,588,109732,109145,0.0,1086.0,588,2019672,minus,TGATGGCGGGAGACACATACGTGCGTGATATCTCTCAGCGTCCAGT...
119,35-260414d,LinJ.35,100.0,559,1,559,193800,194358,0.0,1033.0,559,2019672,plus,CTCAGTGCGTGATATCTCAGCGTCCAGTGCACCCCCACACACACAC...
134,35-315582d,LinJ.35,100.0,578,1,578,248724,249301,0.0,1068.0,578,2019672,plus,GGAGGTGACACACCTCAGCGCATGGTATCCAGGCTCCAATGCCCCC...


In [461]:
data_3["qseqid"].nunique()

68

In [462]:
data_3["qseqid"].value_counts()

qseqid
35A-2072704r    2
35A-2069688r    2
35N-1818131d    2
35N-1830577d    2
35-1554674d     1
               ..
35F-689542r     1
35E-629742r     1
35E-623371r     1
35E-617016d     1
35G-828008d     1
Name: count, Length: 68, dtype: int64

In [463]:
duplicate_qseqids = data_3["qseqid"].value_counts()[lambda x: x > 1].index
filtered_data_3 = data_3[data_3["qseqid"].isin(duplicate_qseqids)]
filtered_data_3

,qseqid,sseqid,pident,length,qstart,qend,sstart,send,evalue,bitscore,qlen,slen,sstrand,sseq
2041,35N-1818131d,LinJ.35,100.0,610,1,610,1771565,1772174,0.000000e+00,1127.0,610,2019672,plus,GATGACGGGAGGGAGGGAACACACCTCAGTGCGGTGGTGTAACAGA...
2042,35N-1818131d,LinJ.35,100.0,610,1,610,1784011,1784620,0.000000e+00,1127.0,610,2019672,plus,GATGACGGGAGGGAGGGAACACACCTCAGTGCGGTGGTGTAACAGA...
2105,35N-1830577d,LinJ.35,100.0,610,1,610,1771565,1772174,0.000000e+00,1127.0,610,2019672,plus,GATGACGGGAGGGAGGGAACACACCTCAGTGCGGTGGTGTAACAGA...
2106,35N-1830577d,LinJ.35,100.0,610,1,610,1784011,1784620,0.000000e+00,1127.0,610,2019672,plus,GATGACGGGAGGGAGGGAACACACCTCAGTGCGGTGGTGTAACAGA...
2677,35A-2069688r,LinJ.35,100.0,498,1,498,8482,7985,0.000000e+00,920.0,498,2019672,minus,CTCTGTATGGGAAAGCCATACAGCCCCCTACTTGCTGACAAATGCG...
2678,35A-2069688r,LinJ.35,100.0,498,1,498,11831,11334,0.000000e+00,920.0,498,2019672,minus,CTCTGTATGGGAAAGCCATACAGCCCCCTACTTGCTGACAAATGCG...
2727,35A-2072704r,LinJ.35,100.0,167,1,167,8151,7985,2.010000e-85,309.0,167,2019672,minus,CTGCGAAGCTGGTGTGGGCAGGGCTTAAGGCGAGGAGGCCGTGCTC...
2728,35A-2072704r,LinJ.35,100.0,167,1,167,11500,11334,2.010000e-85,309.0,167,2019672,minus,CTGCGAAGCTGGTGTGGGCAGGGCTTAAGGCGAGGAGGCCGTGCTC...
